In [11]:
# Imports
import matplotlib.pyplot as plt
import cv2
import torch
import pathlib
import os
from tqdm import tqdm
import numpy as np
from l2cs import Pipeline, render

# Constants 
CWD = pathlib.Path(os.path.abspath(""))
GIT_ROOT = CWD.parent.parent
DATA_DIR = GIT_ROOT / "data" / 'AIED2024'
OUTPUT_DIR = DATA_DIR / 'gaze_vectors'

os.makedirs(OUTPUT_DIR, exist_ok=True)

In [16]:
gaze_pipeline = Pipeline(
    weights=CWD / 'models' / 'L2CSNet_gaze360.pkl',
    arch='ResNet50',
    device=torch.device('cuda'), # or 'gpu'
    include_detector=False
)

In [17]:
dir = DATA_DIR / 'Cam1_SampleData Analysis' / 'C1Sample_cropped_images-20240117T212155Z-001' / 'C1Sample_cropped_images'

for img_fp in tqdm(dir.iterdir(), total=len(list(dir.iterdir()))):
    if img_fp.suffix != '.jpg':
        continue

    assert img_fp.exists()
    frame = cv2.imread(str(img_fp))   

    # Process frame and visualize
    try:
        results = gaze_pipeline.step(frame)
    except Exception as e:
        # print(e)
        continue
    draw = render(frame, results)

    cv2.imshow('frame', draw)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

    # Save the data
    img_name = img_fp.stem
    np.save(OUTPUT_DIR / f'{img_name}_pitch.npy', results.pitch)
    np.save(OUTPUT_DIR / f'{img_name}_yaw.npy', results.yaw)


cv2.destroyAllWindows()

100%|██████████| 1977/1977 [00:37<00:00, 53.12it/s] 


: 